In [1]:
import numpy as np

# Value Loss

In [2]:
def r_gamma(rewards: np.array, gamma):
    r_gamma = 0
    for reward in rewards[:0:-1]:  # Slicing to reverse except the first element
        r_gamma = gamma * (r_gamma + reward)
        print(reward)
    r_gamma += rewards[0]
    return r_gamma

def value_function(state): #TODO: implement this
    return 0

def v_loss(r_gamma, state, deltas):
    return (np.clip(r_gamma, -deltas[1], deltas[2])-value_function(state))**2

# Policy Loss

In [3]:
def policy(state):
    if state == 'Preflop':
        # return np.array([0.5, 0.5, 0, 0, 0])
        return np.array([0, 1, 0, 0, 0])
    elif state == 'Flop':
        # return np.array([0, 0.5, 0.5, 0, 0])
        return np.array([0, 0, 1, 0, 0])
    elif state == 'Turn':
        # return np.array([0, 0, 1/3, 1/3, 1/3])
        return np.array([0, 0, 0, 1, 0])
    elif state == 'River':
        # return np.array([0, 1/2, 1/2, 0, 0])
        return np.array([0, 0, 1, 0, 0])
    return 0

def ratio(old_policy, new_policy, action, state):
    return new_policy(state)[action] / old_policy(state)[action]

def a_gae(results, states, value_function: callable, gamma, lambda_): #This is the advantage function
    def a_k(k):
        a_k = - value_function(states[0]) # First state of the hand
        print(k, results)
        for i in range(k):
            a_k += gamma**(i) * results[i]
        a_k += gamma**(k) * value_function(states[k])
        return a_k
    
    results = results[:-1] # We don't need the last result, we'll use the value function
    a_ks = [lambda_**(k_minus_one) * a_k(k_minus_one+1) for k_minus_one in range(len(results)-1)]
    return (1 - lambda_) * np.sum(a_ks)
    
def tc_loss_function(ratio, advantage, epsilon, deltas): #We compute this for every hand and then average it
    return np.clip(ratio, np.clip(ratio, 1 - epsilon, 1 + epsilon), deltas[0]) * advantage
    

In [4]:
def get_action(policy: callable, state):
    return np.random.choice(len(policy(state)), p=policy(state))

In [5]:
rewards = np.array([-20, -40, 0, -100, 320])
deltas = [3, 160, 160]
print(r_gamma(rewards, 0.999))
print(v_loss(r_gamma(rewards, 0.999), 0, deltas))

320
-100
0
-40
159.06161882032
320
-100
0
-40
25300.598581740778


In [6]:
states = ['Preflop', 'Flop', 'Turn', 'River', 'Showdown']
rewards = [-20, -20, -80, 0, 240] #There should be one more reward than states
deltas = [3, 120, 120]
tc_loss = 0
value_loss = 0
delta2 = 0
delta3 = 10  #Refine this

states_without_showdown = states[:-1]
for i, state in enumerate(states_without_showdown):
    print(states[i:])
    rewards_from_now = rewards[i:]
    states_from_now = states[i:]
    advantage = a_gae(rewards_from_now, states_from_now, value_function, 0.999, 0.99) #I'm not sure if this is correct
    print('Advantage: ', advantage)
    action = get_action(policy, state)
    old_policy = policy
    new_policy = policy
    r = ratio(old_policy, new_policy, action, state)
    delta2 -= rewards[i]
    if i > 1:
        delta3 -= rewards[i-1]
    deltas = [3, delta2, delta3]
    tc_loss += tc_loss_function(r, advantage, 0.2, deltas)
    value_loss += v_loss(r_gamma(rewards_from_now, 0.999), 0, deltas)

tc_loss /= len(states_without_showdown)
value_loss /= len(states_without_showdown)
print('TC loss: ', tc_loss)
print('Value loss: ', value_loss)


['Preflop', 'Flop', 'Turn', 'River', 'Showdown']
1 [-20, -20, -80, 0]
2 [-20, -20, -80, 0]
3 [-20, -20, -80, 0]
Advantage:  -1.7701586040800017
240
0
-80
-20
['Flop', 'Turn', 'River', 'Showdown']
1 [-20, -80, 0]
2 [-20, -80, 0]
Advantage:  -1.1892080000000012
240
0
-80
['Turn', 'River', 'Showdown']
1 [-80, 0]
Advantage:  -0.8000000000000007
240
0
['River', 'Showdown']
Advantage:  0.0
240
TC loss:  -0.939841651020001
Value loss:  3300.0
